In [1]:
import os, re, sys
from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import date
from officelib.xllib import *
import importlib
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter2 as iie
import jpnotebooks.Software.SDLC_traceability_tools.user_tests_parser as utp

UserTestsParser = utp.UserTestsParser
RequirementExtracter = iie.RequirementExtracter

In [2]:
_cache = None
_api = None
_sync_date = None
def _download(**kw):
    global _cache, _api, _sync_date
    if _api is None:
        _api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    _cache = _api.download_issues(project_id="pbssoftware", **kw)
    _sync_date = date.today()
    
def download_issue(id):
    global _cache, _api, _sync_date
    if _api is None:
        _api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    iss = _api.download_issue(id)
    _cache = _cache or {}
    _cache [iss.id] = iss
    
def download(force=False, **kw):
    global _cache, _sync_date
    reason = None
    if _cache is None:
        reason = "cache is None"
    elif force:
        reason = "forced re-cache"
    elif _sync_date is None:
        reason = "can't determine last sync time"
    elif date.today() != _sync_date:
        reason = "cache age > 0 days"
    if reason:
        print("Downloading issues:", reason)
        _download(**kw)
    return _cache.copy()

def load_issues(**kw):
    return download(**kw)

In [3]:
import re
class WebFRSIssuetrackerParser(iie.IssuetrackerParser):
    def __init__(self, types=("URS", "FRS", "SDS", "SWDS")):
        types = list(types) + ["3.0WebFRS"]
        super().__init__(types)
        
    def _get_result_for_line(self, line):
        """ identical to parent function, but checks the type after
        scanning the line to return only 3.0WebFRS items, converted
        to plain FRS items. 
        """
        if not line or line.isspace():
            return self._EMPTY_LINE, None, None, None, True

        m = self._item_match(line)
        if m is None: 
            return self._RAW_LINE, "", "", line.strip(), False
        
        dash1, typ, num, text, dash2 = m.groups()
        if typ != '3.0WebFRS':
            return self._EMPTY_LINE, None, None, None, True
        else:
            typ = 'FRS'
            num = "3." + num
        cancel = dash1 == dash2 and dash1 != ""
        return self._REQ_RESULT, typ, num, text, cancel

In [8]:
rtags = [
    "URS",
    "FRS",
    "SWDS",
    "CS",
    "BUG"
]

ttags = [
    "USR"
]

_ignore = {3194, 3287}
_sprints = {
    'Legacy',
    '3.0',
    '3.0.1',
    '3.0.2',
    '3.0.3',
    '3.0.4',
    '3.0.5',
    '3.0.6',
    '3.0.7'
}
#(globals().get("_cache", None) or {}).clear()
def relevant(i):
    return i.sprint_milestone.name in _sprints and i.id not in _ignore and i.status != "Rejected"

In [25]:
iie = importlib.reload(iie)
utp = importlib.reload(utp)
UserTestsParser = utp.UserTestsParser
RequirementExtracter = iie.RequirementExtracter

force = True
force = False
issues = download(force, status_id="*")
issues2 = list(filter(relevant, issues.values()))
parser = iie.IssuetrackerParser(rtags)
reqs = parser.parse_all(issues2)

parser2 = UserTestsParser()
tests = parser2.parse_excel("C:\\Users\\Nathan\\Documents\\Dropbox\\FRS\\FRS Tests 190226.xlsx")

parser3 = WebFRSIssuetrackerParser(rtags)
reqs2 = parser3.parse_all(issues2)

allitems = reqs + tests + reqs2

def missing_parents(req, ref, reqs):
    if req.type == "TEST":
        return 'ignore'
    else:
        return 'fix'

rex = RequirementExtracter(rtags, ttags, config)
rex.config.missing_parents = missing_parents
rows = rex.extract(allitems)

In [27]:
for i in allitems:
    if i.tag == "FRS4402.2.6"
    5/

In [16]:
def paste(cr, c1, data):
    c2 = c1.Offset(len(data), len(data[0]))
    rng = cr(c1, c2)
    rng.Value = data
    rng.Columns.AutoFit()
    for c in rng.Columns:
        if c.ColumnWidth < 8.11:
            c.ColumnWidth = 8.11
            
xl, wb = xlBook2()
with screen_lock(xl):
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range
    paste(cr, cells(1,1), rows)

In [11]:
xl = Excel()
while True:
    try:
        wb = xl.Workbooks(1)
        wb.Close(False)
    except:
        break
xl.Quit()
del xl